In [ ]:
!pip install requests --quiet

In [ ]:
pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=ffe9dbd4bfa3c6014b02f2243bcf8c8bd49ac1901222f1c68a77fb339a3efdc1
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [19]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

# ReliefWeb API endpoint
url = "https://api.reliefweb.int/v1/disasters"

# ICPAC countries
icpac_countries = ["Burundi", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Rwanda", "Somalia", "South Sudan", "Sudan", "Uganda", "Tanzania"]

# Prepare the payload for the POST request
payload = {
    "filter": {
        "operator": "OR",
        "conditions": []
    },
    "fields": {
        "include": [
            "id", "name", "country", "date", "description"
        ]
    },
    "limit": 100,
    "sort": ["date.created:desc"]
}

# Add conditions for each country
for country in icpac_countries:
    payload["filter"]["conditions"].append({
        "field": "country.name",
        "value": country
    })

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# POST request to fetch the data
response = requests.post(url, headers=headers, json=payload)

# Print response content
print("Response Status Code:", response.status_code)

if response.status_code == 200:
    disaster_data = response.json()
    disasters = disaster_data.get('data', [])

    # Function to extract data from description
    def extract_data_from_description(description):
        data = {
            "total_affected": "Data not available",
            "total_injured": "Data not available",
            "total_missing": "Data not available",
            "total_displaced": "Data not available",
            "economic_impact": "Data not available",
            "total_damaged_houses": "Data not available",
            "total_damaged_schools": "Data not available",
            "total_damaged_health_facilities": "Data not available"
        }

        # Use regular expressions to find the relevant numbers
        total_affected = re.search(r"(\d+,\d+|\d+)\s*people have been affected", description)
        total_injured = re.search(r"(\d+,\d+|\d+)\s*injured", description)
        total_missing = re.search(r"(\d+,\d+|\d+)\s*missing", description)
        total_displaced = re.search(r"(\d+,\d+|\d+)\s*displaced", description)

        if total_affected:
            data["total_affected"] = total_affected.group(1)
        if total_injured:
            data["total_injured"] = total_injured.group(1)
        if total_missing:
            data["total_missing"] = total_missing.group(1)
        if total_displaced:
            data["total_displaced"] = total_displaced.group(1)

        return data

    # Function to scrape additional data from URLs
    def scrape_additional_data(url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                text = soup.get_text()

                # Similar extraction logic from the scraped text
                data = {
                    "total_affected": "Data not available",
                    "total_injured": "Data not available",
                    "total_missing": "Data not available",
                    "total_displaced": "Data not available",
                    "economic_impact": "Data not available",
                    "total_damaged_houses": "Data not available",
                    "total_damaged_schools": "Data not available",
                    "total_damaged_health_facilities": "Data not available"
                }

                total_affected = re.search(r"(\d+,\d+|\d+)\s*people have been affected", text)
                total_injured = re.search(r"(\d+,\d+|\d+)\s*injured", text)
                total_missing = re.search(r"(\d+,\d+|\d+)\s*missing", text)
                total_displaced = re.search(r"(\d+,\d+|\d+)\s*displaced", text)

                if total_affected:
                    data["total_affected"] = total_affected.group(1)
                if total_injured:
                    data["total_injured"] = total_injured.group(1)
                if total_missing:
                    data["total_missing"] = total_missing.group(1)
                if total_displaced:
                    data["total_displaced"] = total_displaced.group(1)

                return data
            else:
                return {}
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")
            return {}

    # Extract the required data
    extracted_data = []
    for disaster in disasters:
        fields = disaster['fields']
        description = fields.get("description", "")
        data_from_description = extract_data_from_description(description)

        # Check for URLs in the description to scrape additional data
        urls = re.findall(r'(https?://[^\s]+)', description)
        additional_data = {}
        for url in urls:
            additional_data = scrape_additional_data(url)
            if additional_data:
                break  # Use the first successful data extraction

        # Merge data from description and additional data
        final_data = {**data_from_description, **additional_data}

        record = {
            "id": disaster["id"],
            "name": fields.get("name"),
            "country": fields.get("country")[0]['name'] if fields.get("country") else "Unknown",
            "date": fields.get("date").get("event"),
            "description": description,
            "total_affected": final_data["total_affected"],
            "total_injured": final_data["total_injured"],
            "total_missing": final_data["total_missing"],
            "total_displaced": final_data["total_displaced"],
            "economic_impact": final_data["economic_impact"],
            "total_damaged_houses": final_data["total_damaged_houses"],
            "total_damaged_schools": final_data["total_damaged_schools"],
            "total_damaged_health_facilities": final_data["total_damaged_health_facilities"]
        }
        extracted_data.append(record)

    # Convert the extracted data to a DataFrame
    new_df = pd.DataFrame(extracted_data)
    print(new_df.head())

else:
    print("Failed to retrieve data. Status code:", response.status_code)
    print("Response Content:", response.content)


Response Status Code: 200
Failed to scrape http://www.rsmcnewdelhi.imd.gov.in/index.php?lang=en#): HTTPSConnectionPool(host='rsmcnewdelhi.imd.gov.inindex.php', port=443): Max retries exceeded with url: /?lang=en (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d97b3578820>: Failed to resolve 'rsmcnewdelhi.imd.gov.inindex.php' ([Errno -2] Name or service not known)"))
Failed to scrape http://www.rsmcnewdelhi.imd.gov.in/index.php?lang=en): HTTPSConnectionPool(host='rsmcnewdelhi.imd.gov.inindex.php', port=443): Max retries exceeded with url: /?lang=en) (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d97b3503ee0>: Failed to resolve 'rsmcnewdelhi.imd.gov.inindex.php' ([Errno -2] Name or service not known)"))
Failed to scrape http://www.met.gov.om/opencms/export/sites/default/dgman/en/home/): HTTPConnectionPool(host='www.met.gov.om', port=80): Max retries exceeded with url: /opencms/export/sites/default/dgman/en/home/) (Caused 

In [20]:
# Convert the extracted data to a DataFrame
new_df = pd.DataFrame(extracted_data)

# Define the file path where you want to save the CSV
csv_file_path = "disaster_data.csv"

# Save the DataFrame to a CSV file
new_df.to_csv(csv_file_path, index=False)

print("Data saved to:", csv_file_path)


Data saved to: disaster_data.csv


In [10]:
!pip install solara altair vega_datasets requests --quiet

In [11]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [17]:
import pandas as pd
import altair as alt
from vega_datasets import data
import streamlit as st

# Create a dataframe from the fetched disaster data
def create_dataframe(disasters):
    records = []
    for disaster in disasters:
        fields = disaster.get('fields', {})

        record = {
            'id': disaster.get('id', 'N/A'),
            'name': disaster.get('name', 'N/A'),
            'country': fields.get('country', [{}])[0].get('name', 'Unknown'),
            'date': fields.get('date', {}).get('created', 'N/A'),
            'description': fields.get('description', 'N/A'),
            'deaths': fields.get('deaths', 0),
            'affected': fields.get('affected', 0),
            'injured': fields.get('injured', 0),
            'missing': fields.get('missing', 0),
            'displaced': fields.get('displaced', 0),
            'economic_cost': fields.get('economic_cost', 0),
            'damaged_houses': fields.get('damaged_houses', 0),
            'damaged_schools': fields.get('damaged_schools', 0),
            'damaged_health_facilities': fields.get('damaged_health_facilities', 0)
        }
        records.append(record)
    return pd.DataFrame(records)

# Create a dataframe
df = create_dataframe(disasters)

# Define a function to create an Altair chart
def create_chart(df):
    world_map = alt.topo_feature(data.world_110m.url, 'countries')

    chart = alt.Chart(world_map).mark_geoshape().encode(
        color=alt.Color('deaths:Q', title='Deaths')
    ).transform_lookup(
        lookup='properties.name',
        from_=alt.LookupData(df, 'country', ['deaths'])
    ).project(
        type='mercator'
    ).properties(
        width=800,
        height=400
    )

    return chart

# Create a Streamlit app
def main():
    st.title("Disaster Dashboard")
    st.header("Map of Disasters")
    chart = create_chart(df)
    st.altair_chart(chart)

if __name__ == "__main__":
    main()


In [ ]:
cd path/to/your/directory
streamlit run disaster_dashboard.py


In [17]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
